## this python script create the correlation between trigger capablity and physical channels

In [1]:
from gensim.models import Word2Vec, KeyedVectors 
from scipy.spatial.distance import cdist # for the Manhattan distance
import json   # for read the JSON file
import stanza # Official Stanford NLP Python Library for Many Human Languages
import re     # for preprocessing the descriptions (necessary for using the word2vec)
import numpy as np

In [2]:
file = open("trigger-action.JSON","r") # read all the capability extrapolated from the apps
trigger_action = json.load(file)
print("tot app analysed:",len(trigger_action.keys()))

tot app analysed: 85


In [3]:
list_trigger_action = []
for key in trigger_action.keys():         # for every app analysed
    set_of_triggers = trigger_action[key] # save the trigger 
    #print("analizzo", key)
    for i in set_of_triggers.keys():      # for every trigger channel
            set_of_action = set_of_triggers[i] # save the set of action of that trigger channel
            for j in range(len(set_of_action)):
                check = any(element == [i, set_of_action[j]] for element in list_trigger_action)
                if not check:
                    list_trigger_action.append([i, set_of_action[j]]) #save capability (the channel) and action
list_trigger_action

[['capability.beacon', ['capability.switch', 'switch.on']],
 ['capability.beacon', ['capability.switch', 'switch.off']],
 ['capability.beacon', ['capability.lock', 'lock.unlock']],
 ['capability.beacon', ['capability.lock', 'lock.lock']],
 ['capability.location', ['capability.switch', 'switch.off']],
 ['capability.app', ['capability.switch', 'switch.off']],
 ['capability.location', ['capability.switch', 'switch.on']],
 ['capability.app', ['capability.switch', 'switch.on']],
 ['capability.presenceSensor', ['sendAction', 'sendNotificationToContacts']],
 ['capability.presenceSensor', ['sendAction', 'sendPush']],
 ['capability.presenceSensor', ['sendAction', 'sendSms']],
 ['capability.presenceSensor',
  ['locationMode', 'locationMode.setLocationMode']],
 ['capability.contactSensor', ['capability.switch', 'switch.on']],
 ['capability.motionSensor', ['capability.switch', 'switch.on']],
 ['capability.button', ['locationMode', 'locationMode.setLocationMode']],
 ['capability.button', ['sendActi

In [4]:
file = open("app-description.JSON","r") # need the desription of the analysed apps
app_description = json.load(file)

In [5]:
file = open("cluster_pysical_channels_FINAL.JSON","r") 
dict_physical_channels = json.load(file)

In [6]:
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English

model = KeyedVectors.load_word2vec_format('/home/volta/Documenti/CCS18_CS/word2vec/GoogleNews-vectors-negative300.bin', binary=True, limit=100000) # load word2vec pre-trained

2021-09-23 08:51:49 INFO: Downloading default packages for language: en (English)...
2021-09-23 08:51:52 INFO: File exists: /home/volta/stanza_resources/en/default.zip.
2021-09-23 08:51:57 INFO: Finished downloading models and saved to /home/volta/stanza_resources.
2021-09-23 08:51:57 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-09-23 08:51:57 INFO: Use device: cpu
2021-09-23 08:51:57 INFO: Loading: tokenize
2021-09-23 08:51:57 INFO: Loading: pos
2021-09-23 08:51:57 INFO: Loading: lemma
2021-09-23 08:51:57 INFO: Loading: depparse
2021-09-23 08:51:58 INFO: Loading: sentiment
2021-09-23 08:51:58 INFO: Loading: ner
2021-09-23 08:51:59 INFO: Done loading processors!


In [9]:
def switch_physical_channel(y_correlation): # conversion between index and channels
    key_list = list(physical_channel_centroids.keys())
    return key_list[y_correlation]

In [18]:
POS_accepted=["NN"]  # part of speech tags accepted ---> deleted: "NNS", "NNP", "NNPS"
physical_ch_capabilities = {"temperature":[],"humidity":[],"illumination":[],"location":[],
                           "motion":[],"smoke":[],"leakage":[]} #dictionary having the relation of physical channels and cap
#for app in trigger_action.keys(): # analyse every app
keywords = [] # keywords found of an app description -> dictionary as "keyword": 300-vector description space
#print("analysing app description", app)
app="beacon-control"
des_analysed = nlp(app_description[app])     # apply Stanford NLP to description
for sent in des_analysed.sentences:
    for word in sent.words:                               # check each word of the description
        if word.xpos in POS_accepted:
            keyword_found = word.text.lower()             # filter: to all lowercase words
            if not keyword_found in keywords:      # if not already saved, store that word of the same desc
                #print(f"found word:{word.text}   \txpos: {word.xpos}\n")
                keywords.append(keyword_found)            # add to the list of keywords

# after finish to check the app description words
clean_keywords = []
for i in range(len(keywords)):                            # for every keyword exstracted
    try:
        model[keywords[i]]                                # check Word2Vec
        clean_keywords.append(keywords[i])
    except KeyError as err:
        print("handling Error: ", err)                    # some words can not be processed by Word2Vec

print(clean_keywords)        
similarity_matrix = {}

if len(clean_keywords) == 0: # case there aren't any NN o NNS keyword inside that app description (strange)
    print(f"-->app [{app}] non presenta nessun NN o NNS")
        
else:
    if len(clean_keywords) == 1: # case there are only 1 NN o NNS keyword inside that app description
        for key in dict_physical_channels.keys(): # check for every physical channel
            similarity_temp = 0
            for channel_NN in range(len(dict_physical_channels[key])):
                similarity_temp += model.similarity(clean_keywords[0],dict_physical_channels[key][channel_NN])
            similarity_matrix[key] = similarity_temp/len(dict_physical_channels[key])
            
    else: # case there are more than 1 NN o NNS keyword inside that app description
        for key in dict_physical_channels.keys(): # check for every physical channel
            similarity_partial = 0
            for i in range(len(clean_keywords)):
                similarity_temp = 0
                for channel_NN in range(len(dict_physical_channels[key])):
                    print("dizionario", dict_physical_channels[key])
                    print(channel_NN)
                    l = dict_physical_channels[key]
                    print(type(l))
                    similarity_temp += model.similarity(clean_keywords[i],dict_physical_channels[key][channel_NN])
                similarity_temp = similarity_temp/len(dict_physical_channels[key])
                similarity_partial += similarity_temp
            similarity_matrix[key] = similarity_partial/len(clean_keywords)
            
    
print(similarity_matrix)

# y_correlation = np.argmin(distance_matrix) % 7            # remember from 0 to 6 ! [7 physical channels]
# physical_channel_tag = switch_physical_channel(y_correlation)
# #print(physical_channel_tag)
# discharge = ["capability.app", "time", "capability.powerMeter", "capability.accelerationSensor"]
# for capability in trigger_action[app].keys():  
#     if not capability in discharge: 
#         print(f"[{app}] use P_Ch:{physical_channel_tag}\tcap:{capability}")
#         physical_ch_capabilities[physical_channel_tag].append(capability)
#     else:
#         print(f"[{app}]--- time, app or powerMeter")

['home', 'phrase', 'door', 'region']
dizionario ['air_conditioner', 'heater', 'humidity', 'moisture', 'sensor', 'sensors', 'sleep', 'temp', 'thermostat', 'weather', 'temperature']
0
<class 'str'>


KeyError: "Key '[' not present"

In [73]:
physical_ch_capabilities

{'Temperature': ['capability.temperatureMeasurement'],
 'Humidity': ['capability.relativeHumidityMeasurement',
  'capability.contactSensor',
  'capability.relativeHumidityMeasurement'],
 'Illumination': ['capability.accelerationSensor',
  'capability.contactSensor',
  'capability.button',
  'capability.motionSensor'],
 'Location': ['capability.beacon',
  'capability.contactSensor',
  'capability.motionSensor',
  'capability.waterSensor',
  'capability.motionSensor',
  'capability.motionSensor',
  'capability.switch',
  'capability.presenceSensor',
  'hub',
  'capability.temperatureMeasurement',
  'capability.temperatureMeasurement',
  'capability.thermostat',
  'capability.contactSensor',
  'capability.contactSensor',
  'capability.contactSensor',
  'capability.motionSensor',
  'capability.contactSensor',
  'capability.presenceSensor',
  'capability.touchSensor',
  'capability.button',
  'capability.contactSensor',
  'capability.accelerationSensor',
  'capability.motionSensor',
  'capa